In [1]:
import jsonstat
import pandas as pd

In [2]:
url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_31_1.jsonstat.json'
data = jsonstat.from_url(url)

print(data)

name:   'Population estimates - local authority based by five year age band'
label:  'Population estimates - local authority based by five year age band'
size: 22200
+-----+-----------+-----------+------+--------+
| pos | id        | label     | size | role   |
+-----+-----------+-----------+------+--------+
| 0   | time      | date      | 37   | time   |
| 1   | geography | geography | 4    | geo    |
| 2   | sex       | sex       | 3    |        |
| 3   | age       | age       | 25   |        |
| 4   | measures  | measures  | 2    | metric |
+-----+-----------+-----------+------+--------+


In [3]:
df = data.to_data_frame('geography')
df.head()

,date,sex,age,measures,Value
geography,,,,,
Wales,1981,Male,All ages,Value,1365100.0
Wales,1981,Male,All ages,Percent,100.0
Wales,1981,Male,Aged under 1 year,Value,18300.0
Wales,1981,Male,Aged under 1 year,Percent,1.3
Wales,1981,Male,Aged 1 - 4 years,Value,70200.0


1 A table showing the male, female and total population in columns, per UK region in rows, as well as the UK total, for the most recent year;

In [4]:
# most recent year is the max year by value
question_1 = df[(df.date == df.date.max()) & (df.age == 'All ages') & (df.measures == 'Value')][['sex','Value']]

question_1.reset_index(inplace=True)
question_1.columns.values[0] = 'region'
question_1.columns.values[-1] = 'value'

In [5]:
def get_england_population_stats(sex):
    population_by_sex = question_1[(question_1.region == 'England and Wales') & (question_1.sex == sex)].iloc[0]['value'] - question_1[(question_1.region == 'Wales') & (question_1.sex == sex)].iloc[0]['value']
    return {'region': 'England', 'sex' : sex, 'value': population_by_sex}

In [6]:
for sex in df.sex.unique():
    question_1 = question_1.append(get_england_population_stats(sex), ignore_index=True)

In [7]:
# drop entries corresponding to 'England and Wales' as this is already considered by the separate regions 
# of England and Wales
question_1 = question_1[question_1.region != 'England and Wales']

In [8]:
question_1 = question_1.append({'region': 'UK', 'sex' : 'Male', 'value': question_1[question_1.sex == 'Male']['value'].sum()}, ignore_index=True)
question_1 = question_1.append({'region': 'UK', 'sex' : 'Female', 'value': question_1[question_1.sex == 'Female']['value'].sum()}, ignore_index=True)
question_1 = question_1.append({'region': 'UK', 'sex' : 'Total', 'value': question_1[question_1.sex == 'Total']['value'].sum()}, ignore_index=True)

In [9]:
question_1.groupby(['region', 'sex']).sum().unstack('sex')

value                        
sex                   Female        Male       Total
region                                              
England           28138300.0  27481100.0  55619400.0
Northern Ireland    950600.0    920200.0   1870800.0
Scotland           2784500.0   2640300.0   5424800.0
UK                33458400.0  32581800.0  66040200.0
Wales              1585000.0   1540200.0   3125200.0

2 Exploratory data analysis to show how the population progressed by regions and age groups;

In [14]:
total_population_by_region = df[(df.age == 'All ages') & (df.sex == 'Total') & (df.measures == 'Value')].reset_index()

# Plot total population by year for each region

In [18]:
total_population_by_region[total_population_by_region.Value.isnull()]

,geography,date,sex,age,measures,Value
2,Northern Ireland,1981,Total,All ages,Value,NaN
6,Northern Ireland,1982,Total,All ages,Value,NaN
10,Northern Ireland,1983,Total,All ages,Value,NaN
14,Northern Ireland,1984,Total,All ages,Value,NaN
18,Northern Ireland,1985,Total,All ages,Value,NaN
22,Northern Ireland,1986,Total,All ages,Value,NaN
26,Northern Ireland,1987,Total,All ages,Value,NaN
30,Northern Ireland,1988,Total,All ages,Value,NaN
34,Northern Ireland,1989,Total,All ages,Value,NaN
38,Northern Ireland,1990,Total,All ages,Value,NaN
